### Perform API calls to create a database of business information for Spas, Restaurants and Hotels through a list of 1300 cities

In [2]:
from yelpapi import YelpAPI

In [3]:
import pandas as pd
import requests
import json
from config import API_Key 
import time 
# from config import bea_api

In [4]:
from pprint import pprint

In [5]:
yelp_api = YelpAPI(API_Key)

In [91]:
#request api data from bureau of economic analysis
bea_url = 'https://apps.bea.gov/api/data'
bea_api = '0BEDA572-87F5-46AB-8614-4949AE2FE5B8'
bea_final_url = f'{bea_url}?&UserID={bea_api}&method=GetData&Datasetname=Regional&TableName=CAINC1&LineCode=3&GeoFIPS=COUNTY&Year=2014&ResultFormat=JSON'
bea_list_url=f'https://apps.bea.gov/api/data?&UserID={bea_api}&method=GETDATASETLIST&'
bea_list = requests.get(bea_list_url).json()
bea_data = requests.get(bea_final_url).json()

In [93]:
data = bea_data['BEAAPI']['Results']['Data']

In [94]:
#drop columns and save to csv
PCI = pd.DataFrame(data)
PCI.drop(['Code', 'GeoFips', 'CL_UNIT', 'UNIT_MULT', 'NoteRef'], axis = 1, inplace=True)
PCI.to_csv("PCI.csv")
PCI = pd.read_csv('PCI.csv')

In [31]:
PCI

,Unnamed: 0,GeoName,TimePeriod,DataValue
0,0,"Autauga, AL",2014,"36,699"
1,1,"Baldwin, AL",2014,"39,605"
2,2,"Barbour, AL",2014,"29,920"
3,3,"Bibb, AL",2014,"28,255"
4,4,"Blount, AL",2014,"31,415"
...,...,...,...,...
3133,3133,"Sweetwater, WY",2014,"49,061"
3134,3134,"Teton, WY",2014,"200,044"
3135,3135,"Uinta, WY",2014,"39,275"
3136,3136,"Washakie, WY",2014,"44,349"


In [26]:
#read html data for population(old, newer used census)
pop_html = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population')

In [27]:
pop_db = pop_html[4]
pop_db.drop(['Location', '2016 population density', '2016 population density.1', '2018rank'], axis = 1, inplace=True)
pop_db = pop_db.rename(columns={'State[c]':'State'})

In [32]:
pop_db['City'] = pop_db['City'].apply(lambda x: x.partition('[')[0])

In [33]:
pop_db.to_csv('pop_db.csv')

In [156]:
PCI['GeoName'].p

0          Autauga, AL
1          Baldwin, AL
2          Barbour, AL
3             Bibb, AL
4           Blount, AL
             ...      
3133    Sweetwater, WY
3134         Teton, WY
3135         Uinta, WY
3136      Washakie, WY
3137        Weston, WY
Name: GeoName, Length: 3138, dtype: object

In [37]:
#seperate city and state into 2 columns
PCI['City'] = PCI['GeoName'].apply(lambda x: x.partition(',')[0])
PCI['State'] = PCI['GeoName'].apply(lambda x: x.partition(',')[2])
PCI.drop(['GeoName', 'TimePeriod'], axis=1, inplace=True)


In [40]:
#read in state abbr. into df and merge with pop
st_abbr = pd.read_csv('st_abbr.csv')
st_abbr.drop(['Abbrev'], axis=1, inplace=True)
population = st_abbr.merge(pop_db, on='State')
population.drop(['State'], axis=1, inplace=True)
population = population.rename(columns={'Code':'State'})
final_db = population.merge(PCI, on='City')

In [48]:
#delete empty spaces before and after State abbr
PCI['State'] = PCI['State'].apply(lambda x: x.strip())

In [50]:
final_db = population.merge(PCI, on=['City', 'State'])

In [53]:
final_db.to_csv('merged_data.csv')

In [5]:
final_db = pd.read_csv("merged_data.csv")

In [6]:

final_before = pd.read_csv('final_before_yelp.csv')

In [7]:
#loop through merged database to gather a list of City, St for API call
yelp_list_st = []
yelp_list_city = []
for row in final_before.iterrows():
    yelp_list_city.append(row[1][2])
    yelp_list_st.append(row[1][1])


In [8]:
len(yelp_list_city)

1223

In [10]:
#loop through results and create dataframe for desired info

def getresults(location, terms):
    
    time.sleep(1.5)
    name = []
    cat = []
    price = []
    rating = []
    review = []
    state = []
    city = []
    bus_Type = []
    total_list = []
    
    try:
        results = yelp_api.search_query(term=terms, location=location)
        time.sleep(1)
    except: 
        print(f'no listing for {location} and {terms}')
        dict1 = {}
        return dict1
     
    #loop through creating offsets of 50 for each request     
    offset = 0
    total = results['total']
    if total > 500:
        x = 10
    elif total > 0 and total < 50: 
        x = 1
    else:
        x = round(total/50) #calculate how many offset calls to make
    
    for i in range(0,x):
        if i == 0:
            offset = 0
        else: 
            offset = offset + 50 #increase each offset by 50 for each iteration
        
        try:
            results = yelp_api.search_query(term=terms, location=location, offset=offset) 
            time.sleep(1)
            
            #get results and append to list
            for rest in results['businesses']:
                name.append(rest['name'])
                cat.append(rest['categories'][0]['title'])
                state.append(location.split(',')[1])
                city.append(location.split(',')[0])
                bus_Type.append(terms)
                total_list.append(total)

                try:
                    price.append(f"{rest['price']}")
                except: 
                    price.append('0')
                try:
                    rating.append(rest['rating'])
                except: 
                    rating.append('0')  
                try:                
                    review.append(rest['review_count'])
                except:
                    review.append('0')
            
            
            dict1 = {'State': state,
                'City': city,
                'name': name,
                'cat': cat,
                'price': price,
                'rating': rating, 
                'review': review,
                'BusinessType':bus_Type,
                'Total': total_list}
                                 
        except: 
            print(f'error reached at {location} and {terms} offset {offset}')
            try: 
                dict1
            except:
                dict1 = {}
            finally:
                return dict1
    try:                             
        return dict1                        
    except UnboundLocalError:
        dict1={}
        return dict1

In [11]:
#main loop to call query function
resultx = []

for i in range(0, len(yelp_list_city)):
    location = f'{yelp_list_city[i].lower()}, {yelp_list_st[i].lower()}'
    result = getresults(location, 'hotel')
    resultx.append(result)
        
        

error reached at charles, md and hotel offset 450
error reached at clare, mi and hotel offset 0
error reached at bay, mi and hotel offset 100
error reached at isabella, mi and hotel offset 0
error reached at leelanau, mi and hotel offset 100
no listing for madison, oh and hotel
error reached at snyder, pa and hotel offset 350
error reached at monroe, pa and hotel offset 400
error reached at pike, pa and hotel offset 300
error reached at lycoming, pa and hotel offset 150
error reached at kershaw, sc and hotel offset 300
error reached at alexandria, va and hotel offset 200
error reached at clay, wv and hotel offset 50


In [12]:
#create database from list of dictionaries
b = pd.DataFrame(resultx[0])
for i in range(1, len(resultx)):
    try:
        c = pd.DataFrame(resultx[i])
        b = b.append(c)
    except ValueError:
        print(f'error at {i}')

error at 426
error at 466
error at 475
error at 906
error at 919
error at 922
error at 948
error at 973
error at 1123
error at 1157


In [13]:
b


,State,City,name,cat,price,rating,review,BusinessType,Total
0,al,butler,The Mason,Hotels,$$,5.0,3,hotel,53
1,al,butler,The Mason,Hotels,0,5.0,1,hotel,53
2,al,butler,Western Inn,Hotels,0,3.0,1,hotel,53
3,al,butler,Hampton Inn Jackson,Hotels,0,4.5,7,hotel,53
4,al,butler,Drury Inn & Suites - Meridian,Hotels,$$,4.0,42,hotel,53
...,...,...,...,...,...,...,...,...,...
35,id,boise,Ameritel Inn,Hotels,0,5.0,1,hotel,85
36,id,boise,Airport Inn,Hotels,0,3.5,3,hotel,85
37,id,boise,Bogus Basin Recreation Area,Hotels,0,5.0,1,hotel,85
38,id,boise,The Sapphire Room At The Riverside Hotel,Jazz & Blues,$$,4.0,11,hotel,85


In [14]:
b.to_csv('finalhoteldata.csv')